In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install

In [2]:
import pandas as pd
import numpy as np

In [9]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ANN/churn modeling/Churn_Modelling.csv')

In [10]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [12]:
#create dummy variable
X = pd.get_dummies(X,drop_first=True)

In [13]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [14]:
#now scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [15]:
#split the datset in train test 
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [17]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.activations import relu,sigmoid


In [25]:
def create_model(layers,activation):
  model = Sequential()
  for i , nodes in enumerate(layers):
    if i==0:
      model.add(Dense(units =nodes,input_dim = X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(units=nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  #last layer
  model.add(Dense(units =1,kernel_initializer='glorot_uniform',activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [26]:
model = KerasClassifier(build_fn=create_model,verbose=0)

In [29]:
layers = [(40,),(40,20),(20,20),(45,30,15)]
activations = ['sigmoid','relu']
param_grid = dict(layers = layers,activation = activations,batch_size=[128,256],epochs=[30])
grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

In [30]:
grid_result = grid.fit(X_train,y_train)

In [31]:
print(grid_result.best_score_,grid_result.best_params_)

0.8565000057220459 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}


In [32]:
pred_y = grid.predict(X_test)
y_pred = (pred_y>0.5)

In [33]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
score = accuracy_score(y_test,y_pred)

In [34]:
cm

array([[1534,   61],
       [ 206,  199]])

In [35]:
score

0.8665

In [36]:
#as it shows after hyperparameter tuning score is imporved from 0.83 to 0.86